In [13]:
import pandas as pd 
from requests_html import HTMLSession
import os
import requests
from lxml import html 
import csv
from datetime import date

In [14]:
today = date.today()
# most recent trading day used for testing 
today_sub = '2022-04-08'
session = HTMLSession()
url = 'https://sec.report/Senate-Stock-Disclosures'
r = session.get(url)
print(r)

<Response [200]>


In [15]:
table = r.html.find('table')[0]
rows = table.find('tr')
trades = rows[1:]
n = len(trades)

In [16]:
def arr_to_dict(lst):
    it = iter(lst)
    res_dict = dict(zip(it,it))
    return res_dict 
def value_to_ints(value):
    bad_chars = [
        ',','$','-'
    ]
    for c in bad_chars:
        value = value.replace(c,'')
    low, high = [
        int(x) for x in (value.split('  ', 1))
    ]
    return [low,high]

In [17]:
all_trades = []
l1_head = [
    'file date', 'trade', 'senator'
]
l2_head = [
    'trade type', 'value'
]
current = True
while current:
    for i in range(0,n,2):
        trade = []
        l1_elements = trades[i].find('td')
        l2_elements = trades[i+1].find('td')[:-1]
        for h,e in zip(l1_head, l1_elements):
            trade.append(h)
            trade.append(e.text)
        for h,e in zip(l2_head, l2_elements):
            trade.append(h)
            trade.append(e.text)
        trade[1] = trade[1][:10]
        # today_sub used #
        if str(today_sub) != trade[1]:
            current = False
            break
        trade[7] = trade[7].split('\n', 1)[0]
        trade[9] = value_to_ints(trade[9])
        trade = arr_to_dict(trade)
        all_trades.append(trade)

In [18]:
# scans for trades with value over certain amount (???)
for t in all_trades:
    if t['value'][1] > 15001:
        print(t['trade'], t['trade type'], t['value'])

Humacyte, Inc. - Common Stock [HUMA] Purchase [15001, 50000]
Microsoft Corporation - Common Stock Option Type: Short Sale Strike price: $265.00 Expires: 10/21/2022 [MSFT] Sale (Full) [15001, 50000]
ChannelAdvisor Corporation Common Stock [ECOM] Purchase [15001, 50000]
Nu Holdings Ltd. Class A Ordinary Shares [NU] Purchase [15001, 50000]
ChannelAdvisor Corporation Common Stock [ECOM] Purchase [50001, 100000]
ARKK Option Type: Short Sale Strike price: $65.00 Expires: 09/16/2022 Sale (Full) [15001, 50000]


# To do 

- decide on filter metric for determning high-value trades (just buys?)
- set up email (delivery method?) to send high-value trades from this address to me (and others?)
- create script to run program at certain times (twice a day? noon and at market close?)
- monitor senate disclosure site (publish on trades file date? always within a window of file date? publish at certain time?)
- search ticker in google and send top news article links with notification? mkt watch link using ticker?
- filter out commodity/debt trades